# S3 What are adapation? Bursting?
##### Adapted from a similar GENESIS tutorial (Bower and Beeman, 2007) by Charlie Franklin and Henry Chen, converted to notebook by Ziao Chen and Zhenru Chen

### Model Features:
  - Two compartments, soma and axon, with different sets of currents, i.e., a more realistic model of the neuron  
  - SOMA currents: I<sub>CaS</sub>, I<sub>A</sub>, I<sub>CaT</sub>, I<sub>KCa</sub>, I<sub>Kd</sub>, I<sub>leak</sub>  
  - Axon currents: I<sub>Na</sub>, I<sub>Kd</sub>, I<sub>leak</sub>  
  - I<sub>CaS</sub> and I<sub>A</sub> counteract each other and their relative ratio decides whether the cell would be an endogenous or forced burster (see exercises below related to this concept).  
  - The fact that axon has the action potential currents makes the spikes look small in the soma. Why? The job of the soma then is to generate the depolarization using primarily CaS and A, and a depolarized soma then depolarizes the axon and when this axon compartment's voltage crossed the threshold, you get spikes in the axon. These spikes then ‘back propagate’ to the soma, in addition to traveling down the axon to the next neuron.

#### Before running the simulation, you need to compile the mod files only once for the first time. In Linux or MacOS, run the cell below to compile. In Windows, you need to run 'mknrndll' instead and select current folder to compile. 

In [ ]:
import os
print(os.system('nrnivmodl')) # compile modfiles. Return 0 for success, 1 for failure.

### Run the codes below and answer the 9 questions at the end.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to Hide/Show the code."></form>''')

In [2]:
from IPython.display import Javascript
Javascript('IPython.notebook.execute_cells_below()')

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
from math import pi
from neuron import h
h.load_file('stdrun.hoc')

# Simulation parameters
h.dt = 0.025 # time step (resolution) of the simulation in ms
h.v_init= -50 # initial membrane potential in mV

# Create the soma section and define the parameters
soma = h.Section(name='soma')
soma.diam = 1000;
soma.L = 1e5/pi # micrometers
soma.cm = 9e-3 # membrane capacitance uF/cm2
soma.Ra = 10000  # ohm-cm

soma.insert('leak'); soma.insert('cas'); soma.insert('cat')
soma.insert('kA'); soma.insert('kca'); soma.insert('kdr')
soma.insert('capool')
soma.el_leak=-50; soma.eca = 120; soma.ek = -80 # mV
soma.cainf_capool = 500e-6; soma.casi = soma.cainf_capool # mM

# Create the axon section and define the parameters
axon = h.Section(name='axon')
axon.diam = 1000 # micrometers
axon.L = 1e5/pi # micrometers
axon.cm = 1.5e-3 # uF/cm2
axon.Ra = 10000  # ohm-cm

axon.insert('leak'); axon.insert('na'); axon.insert('kdr')
axon.el_leak = -60; axon.ena = 50; axon.ek = -80 # mV

# Connect axon to the soma
axon.connect(soma(1),0)
h.topology()

|-|       soma(0-1)
   `|       axon(0-1)



1.0

In [4]:
# Current clamp
ccl = h.IClamp(soma(0.5))

# Define vectors for recording variables
t_vec = h.Vector(); s_v_vec = h.Vector()
s_il_vec = h.Vector(); s_ikd_vec = h.Vector()
icas_vec = h.Vector(); icat_vec = h.Vector()
ia_vec = h.Vector(); ikca_vec = h.Vector()
a_v_vec = h.Vector(); a_il_vec = h.Vector()
ina_vec = h.Vector(); a_ikd_vec = h.Vector()
casi_vec = h.Vector()

# Record the variables
t_vec.record(h._ref_t); s_v_vec.record(soma(0.5)._ref_v)
s_il_vec.record(soma(0.5)._ref_il_leak); s_ikd_vec.record(soma(0.5)._ref_ikd_kdr)
icas_vec.record(soma(0.5)._ref_icas_cas); icat_vec.record(soma(0.5)._ref_icat_cat)
ia_vec.record(soma(0.5)._ref_ia_kA); ikca_vec.record(soma(0.5)._ref_ikca_kca)
a_v_vec.record(axon(0.5)._ref_v); a_il_vec.record(axon(0.5)._ref_il_leak)
ina_vec.record(axon(0.5)._ref_ina); a_ikd_vec.record(axon(0.5)._ref_ikd_kdr)
casi_vec.record(soma(0.5)._ref_casi)

def plot_variables():
    plt.figure(figsize=(13,20))
    # Soma membrane potential
    plt.subplot(5,1,1)
    plt.plot(t_vec, s_v_vec,'k')
    plt.xlim(0,h.tstop); plt.ylim(-100,50)
    plt.ylabel('mV'); plt.legend(['soma Vm'],loc=1)
    # Soma channel currents
    plt.subplot(5,1,2)
    plt.plot(t_vec,s_il_vec,'g'); plt.plot(t_vec,s_ikd_vec,'b')
    plt.plot(t_vec,icas_vec,'brown'); plt.plot(t_vec,icat_vec,'orange')
    plt.plot(t_vec,ia_vec,'purple'); plt.plot(t_vec,ikca_vec,'y')
    plt.xlim(0,h.tstop)
    plt.ylabel('mA/cm2'); plt.legend(['il','icas','icat','ia','ikca','ikd'],title='soma',loc=1)
    # Axon membrane potential
    plt.subplot(5,1,3)
    plt.plot(t_vec,a_v_vec ,'k')
    plt.xlim(0,h.tstop); plt.ylim(-100,50)
    plt.ylabel('mV'); plt.legend(['axon Vm'],loc=1)
    # Axon channel currents
    plt.subplot(5,1,4)
    plt.plot(t_vec,a_il_vec ,'g')
    plt.plot(t_vec,ina_vec ,'r')
    plt.plot(t_vec,a_ikd_vec ,'b')
    plt.xlim(0,h.tstop);
    plt.ylabel('mA/cm2'); plt.legend(['il','ina','ikd'],title='axon',loc=1)
    # Ca pool concentration
    plt.subplot(5,1,5)
    plt.plot(t_vec,casi_vec,'c')
    plt.xlim(0,h.tstop); plt.xlabel('time (ms)')
    plt.ylabel('mM'); plt.legend(['Ca pool'],loc=1)
    plt.show()

def activemodel(tstop,soma_gl,gcas,gcat,gkA,gkca,soma_gkdr,taucas,axon_gl,axon_gna,axon_gkdr,amp,dur,delay,fig):
    soma.glbar_leak = soma_gl
    soma.gcasbar_cas = gcas
    soma.gcatbar_cat = gcat
    soma.gkAbar_kA = gkA
    soma.gkcabar_kca = gkca
    soma.gkdrbar_kdr = soma_gkdr
    soma.taucas_capool = taucas
    axon.glbar_leak = axon_gl
    axon.gnabar_na = axon_gna
    axon.gkdrbar_kdr = axon_gkdr
    ccl.amp = amp
    ccl.dur = dur
    ccl.delay = delay
    
    h.tstop = tstop
    h.run()
    plt.close('all')
    plot_variables()

In [5]:
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display
%matplotlib inline

# default setting
tstop = 1000 # how long to run the simulation in ms
soma_gl = 0.000045e-3 # S/cm2
gcas = 0.055e-3 # S/cm2
gcat = 0.0552e-3 # S/cm2
gkA = 0.200e-3 # S/cm2
gkca = 0.500e-3 # S/cm2
soma_gkdr = 1.890e-3 # S/cm2
taucas = 303 # ms
axon_gl = 0.0000018e-3 # S/cm2
axon_gna = 0.300e-3 # S/cm2
axon_gkdr = 0.0525e-3 # S/cm2
amp = 0 # amplitude in nA
dur = 600 # duration in ms
delay = 200 # delay in ms

w_reset = widgets.Button(description='Reset',icon='history',button_style='primary')
w_fig = widgets.ToggleButton(value=False,description='Interactive plot',icon='window-restore',button_style='success')
w_tstop = widgets.BoundedFloatText(value=tstop,min=1,max=5e4,description='Tstop (ms)')
w_s_gl = widgets.BoundedFloatText(value=soma_gl,min=0,max=1,step=1e-9)
w_gcas = widgets.BoundedFloatText(value=gcas,min=0,max=1,step=1e-7)
w_gcat = widgets.BoundedFloatText(value=gcat,min=0,max=1,step=1e-7)
w_gkA = widgets.BoundedFloatText(value=gkA,min=0,max=1,step=1e-6)
w_gkca = widgets.BoundedFloatText(value=gkca,min=0,max=1,step=1e-4)
w_s_gkdr = widgets.BoundedFloatText(value=soma_gkdr,min=0,max=1,step=1e-4)
w_taucas = widgets.BoundedFloatText(value=taucas,min=0,max=1e4)
w_a_gl = widgets.BoundedFloatText(value=axon_gl,min=0,max=1,step=1e-10)
w_a_gna = widgets.BoundedFloatText(value=axon_gna,min=0,max=1,step=1e-5)
w_a_gkdr = widgets.BoundedFloatText(value=axon_gkdr,min=0,max=1,step=1e-6)
w_amp = widgets.FloatText(value=amp)
w_dur = widgets.FloatText(value=dur)
w_delay = widgets.FloatText(value=delay)

def reset_default(*args):
    w_tstop.value = tstop; w_s_gl.value = soma_gl;
    w_gcas.value = gcas; w_gcat.value = gcat
    w_gkA.value = gkA; w_gkca.value = gkca; w_s_gkdr.value = soma_gkdr
    w_taucas.value = taucas; w_a_gl.value = axon_gl
    w_a_gna.value = axon_gna; w_a_gkdr.value = axon_gkdr
    w_amp.value = amp; w_dur.value = dur; w_delay.value = delay
w_reset.on_click(reset_default)

def interactive_fig(*arg):
    if w_fig.value:
        w_fig.icon = 'window-maximize'; w_fig.description='Inline plot'
        %matplotlib qt
        %matplotlib qt
    else:
        w_fig.icon = 'window-restore'; w_fig.description='Interactive plot'
        %matplotlib inline
w_fig.observe(interactive_fig,'value')

between = Layout(justify_content='space-between')
around = Layout(justify_content='space-around')
labels = ['gL:','gCaS:','gCaT:','gKA:','gKCa:','gKdr:','tauCa (ms):',
          'Amplitude (nA):','Duration (ms):','Delay (ms):','gL:','gNa:','gKdr:']
Labels = [Label(L) for L in labels]

ui = VBox([ HBox([w_reset,w_fig]), w_tstop, HBox([
            VBox([ Label(r'\( Soma\ channels\ conductance\ (S/cm^{2}) \)'),
                  HBox([ VBox(Labels[:7]),VBox([w_s_gl,w_gcas,w_gcat,w_gkA,w_gkca,w_s_gkdr,w_taucas]) ], layout=between) ]),
            VBox([ Label(r'\( Current\ clamp \)'),
                  HBox([ VBox(Labels[7:10]),VBox([w_amp,w_dur,w_delay]) ], layout=between),
                   Label(r'\( Axon\ channels\ conductance\ (S/cm^{2}) \)'),
                  HBox([ VBox(Labels[-3:]),VBox([w_a_gl,w_a_gna,w_a_gkdr]) ], layout=between) ]) ], layout=around) ])
out = interactive_output(activemodel,{'tstop':w_tstop,'soma_gl':w_s_gl,'gcas':w_gcas,'gcat':w_gcat,
                    'gkA':w_gkA,'gkca':w_gkca,'soma_gkdr':w_s_gkdr,'taucas':w_taucas,'axon_gl':w_a_gl,
                    'axon_gna':w_a_gna,'axon_gkdr':w_a_gkdr,'amp':w_amp,'dur':w_dur,'delay':w_delay,'fig':w_fig})

display(ui,out)

Output()

### Questions:
1. Run the simulation and see which currents are active during different phases of the burst. Which currents are responsible for which dynamics of the burst?  
<br>
2. Turn off the I<sub>KCa</sub> I<sub>CaS</sub> and I<sub>CaT</sub> currents by setting their conductances to zero. What will this do to the cell? Can you see spikes? Why not? What can you do to get spiking behavior?  
<br>
3. Turn off I<sub>Na</sub> by setting the conductance g<sub>Na</sub> to zero. Do slow wave oscillations still occur? Explain.  
<br>
4. A bursting cell has two sets of currents, one set responsible for generating action potentials and one for generating the slow wave oscillations. Describe these sets in detail below and explain their functions in creating the bursts.  
<br>  
_<u>(change Tstop to 5000 ms for the following set of questions)<u>_
5. Observe the Ca concentration. What role does the Ca concentration play in the burst dynamics?  
<br>
6. Set the I<sub>KCa</sub> conductance to zero. What happens to the burst? Why?  
<br>
7. Observe and explain the effects of increasing the I<sub>A</sub> conductance.  
<br>
8. Observe and explain the effects of increasing the I<sub>CaS</sub> conductance.  
<br>
9. What happens if g<sub>A</sub> and g<sub>CaS</sub> are increased or decreased together?